In [1]:
# detect

import argparse
import sys
import time
from pathlib import Path

import cv2
import torch
import torch.backends.cudnn as cudnn

# FILE = Path(__file__).absolute()
# sys.path.append(FILE.parents[0].as_posix())  # add yolov5/ to path

from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, colorstr, non_max_suppression, \
    apply_classifier, scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path, save_one_box
from utils.plots import colors, plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized

In [3]:
import os
os.listdir('../YOLO(2)')

['data.yaml',
 'yolov5',
 'train',
 '.ipynb_checkpoints',
 'valid',
 'YOLOv5_with HomePointData(3).ipynb',
 'README.roboflow.txt']

In [4]:
from utils.general import *; _ = kmean_anchors(path='../YOLO(2)/data.yaml', n=1, img_size=640, thr=0.5, gen=1000, verbose=True)

NameError: name 'kmean_anchors' is not defined

In [20]:
@torch.no_grad()
def run(weights='yolov5s_jihyun.pt',  # model.pt path(s)
        source='data/images',  # file/dir/URL/glob, 0 for webcam
        imgsz=640,  # inference size (pixels)
        conf_thres=0.25,  # confidence threshold
        iou_thres=0.45,  # NMS IOU threshold
        max_det=1000,  # maximum detections per image
        device='',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
        view_img=False,  # show results
        save_txt=False,  # save results to *.txt
        save_conf=False,  # save confidences in --save-txt labels
        save_crop=False,  # save cropped prediction boxes
        nosave=False,  # do not save images/videos
        classes=None,  # filter by class: --class 0, or --class 0 2 3
        agnostic_nms=False,  # class-agnostic NMS
        augment=False,  # augmented inference
        update=False,  # update all models
        project='runs/detect',  # save results to project/name
        name='exp',  # save results to project/name
        exist_ok=False,  # existing project/name ok, do not increment
        line_thickness=3,  # bounding box thickness (pixels)
        hide_labels=False,  # hide labels
        hide_conf=False,  # hide confidences
        half=False,  # use FP16 half-precision inference
        ):
    save_img = not nosave and not source.endswith('.txt')  # save inference images
    webcam = source.isnumeric() or source.endswith('.txt') or source.lower().startswith(
        ('rtsp://', 'rtmp://', 'http://', 'https://'))

    # Directories
    save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Initialize
    set_logging()
    device = select_device(device)
    half &= device.type != 'cpu'  # half precision only supported on CUDA

    # Load model
    model = attempt_load(weights, map_location=device)  # load FP32 model
    stride = int(model.stride.max())  # model stride
    imgsz = check_img_size(imgsz, s=stride)  # check image size
    names = model.module.names if hasattr(model, 'module') else model.names  # get class names
    if half:
        model.half()  # to FP16

    # Second-stage classifier
    classify = False
    if classify:
        modelc = load_classifier(name='resnet50', n=2)  # initialize
        modelc.load_state_dict(torch.load('resnet50.pt', map_location=device)['model']).to(device).eval()

    # Set Dataloader
    vid_path, vid_writer = None, None
    if webcam:
        view_img = check_imshow()
        cudnn.benchmark = True  # set True to speed up constant image size inference
        dataset = LoadStreams(source, img_size=imgsz, stride=stride)
    else:
        dataset = LoadImages(source, img_size=imgsz, stride=stride)

    # Run inference
    if device.type != 'cpu':
        model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once
    t0 = time.time()
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = time_synchronized()
        pred = model(img, augment=augment)[0]

        # Apply NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        t2 = time_synchronized()

        # Apply Classifier
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            if webcam:  # batch_size >= 1
                p, s, im0, frame = path[i], f'{i}: ', im0s[i].copy(), dataset.count
            else:
                p, s, im0, frame = path, '', im0s.copy(), getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # img.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # img.txt
            s += '%gx%g ' % img.shape[2:]  # print stringconda conda install
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            imc = im0.copy() if save_crop else im0  # for save_crop
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
                        with open(txt_path + '.txt', 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    if save_img or save_crop or view_img:  # Add bbox to image
                        c = int(cls)  # integer class
                        label = None if hide_labels else (names[c] if hide_conf else f'{names[c]} {conf:.2f}')
                        plot_one_box(xyxy, im0, label=label, color=colors(c, True), line_thickness=line_thickness)
                        if save_crop:
                            save_one_box(xyxy, imc, file=save_dir / 'crops' / names[c] / f'{p.stem}.jpg', BGR=True)

            # Print time (inference + NMS)
            print(f'{s}Done. ({t2 - t1:.3f}s)')

            # Stream results
            if view_img:
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'image':
                    cv2.imwrite(save_path, im0)
                else:  # 'video' or 'stream'
                    if vid_path != save_path:  # new video
                        vid_path = save_path
                        if isinstance(vid_writer, cv2.VideoWriter):
                            vid_writer.release()  # release previous video writer
                        if vid_cap:  # video
                            fps = vid_cap.get(cv2.CAP_PROP_FPS)
                            w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                            h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        else:  # stream
                            fps, w, h = 30, im0.shape[1], im0.shape[0]
                            save_path += '.mp4'
                        vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                    vid_writer.write(im0)

    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        print(f"Results saved to {save_dir}{s}")

    if update:
        strip_optimizer(weights)  # update model (to fix SourceChangeWarning)

    print(f'Done. ({time.time() - t0:.3f}s)')

In [20]:
@torch.no_grad()
def run(weights='yolov5s_jihyun.pt',  # model.pt path(s)
        source='data/images',  # file/dir/URL/glob, 0 for webcam
        imgsz=640,  # inference size (pixels)
        conf_thres=0.25,  # confidence threshold
        iou_thres=0.45,  # NMS IOU threshold
        max_det=1000,  # maximum detections per image
        device='',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
        view_img=False,  # show results
        save_txt=False,  # save results to *.txt
        save_conf=False,  # save confidences in --save-txt labels
        save_crop=False,  # save cropped prediction boxes
        nosave=False,  # do not save images/videos
        classes=None,  # filter by class: --class 0, or --class 0 2 3
        agnostic_nms=False,  # class-agnostic NMS
        augment=False,  # augmented inference
        update=False,  # update all models
        project='runs/detect',  # save results to project/name
        name='exp',  # save results to project/name
        exist_ok=False,  # existing project/name ok, do not increment
        line_thickness=3,  # bounding box thickness (pixels)
        hide_labels=False,  # hide labels
        hide_conf=False,  # hide confidences
        half=False,  # use FP16 half-precision inference
        ):
    save_img = not nosave and not source.endswith('.txt')  # save inference images
    webcam = source.isnumeric() or source.endswith('.txt') or source.lower().startswith(
        ('rtsp://', 'rtmp://', 'http://', 'https://'))

    # Directories
    save_dir = increment_path(Path(project) / name, exist_ok=exist_ok)  # increment run
    (save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

    # Initialize
    set_logging()
    device = select_device(device)
    half &= device.type != 'cpu'  # half precision only supported on CUDA

    # Load model
    model = attempt_load(weights, map_location=device)  # load FP32 model
    stride = int(model.stride.max())  # model stride
    imgsz = check_img_size(imgsz, s=stride)  # check image size
    names = model.module.names if hasattr(model, 'module') else model.names  # get class names
    if half:
        model.half()  # to FP16

    # Second-stage classifier
    classify = False
    if classify:
        modelc = load_classifier(name='resnet50', n=2)  # initialize
        modelc.load_state_dict(torch.load('resnet50.pt', map_location=device)['model']).to(device).eval()

    # Set Dataloader
    vid_path, vid_writer = None, None
    if webcam:
        view_img = check_imshow()
        cudnn.benchmark = True  # set True to speed up constant image size inference
        dataset = LoadStreams(source, img_size=imgsz, stride=stride)
    else:
        dataset = LoadImages(source, img_size=imgsz, stride=stride)

    # Run inference
    if device.type != 'cpu':
        model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once
    t0 = time.time()
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = time_synchronized()
        pred = model(img, augment=augment)[0]

        # Apply NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        t2 = time_synchronized()

        # Apply Classifier
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            if webcam:  # batch_size >= 1
                p, s, im0, frame = path[i], f'{i}: ', im0s[i].copy(), dataset.count
            else:
                p, s, im0, frame = path, '', im0s.copy(), getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # img.jpg
            txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # img.txt
            s += '%gx%g ' % img.shape[2:]  # print stringconda conda install
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            imc = im0.copy() if save_crop else im0  # for save_crop
            if len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                # Write results
                for *xyxy, conf, cls in reversed(det):
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
                        with open(txt_path + '.txt', 'a') as f:
                            f.write(('%g ' * len(line)).rstrip() % line + '\n')

                    if save_img or save_crop or view_img:  # Add bbox to image
                        c = int(cls)  # integer class
                        label = None if hide_labels else (names[c] if hide_conf else f'{names[c]} {conf:.2f}')
                        plot_one_box(xyxy, im0, label=label, color=colors(c, True), line_thickness=line_thickness)
                        if save_crop:
                            save_one_box(xyxy, imc, file=save_dir / 'crops' / names[c] / f'{p.stem}.jpg', BGR=True)

            # Print time (inference + NMS)
            print(f'{s}Done. ({t2 - t1:.3f}s)')

            # Stream results
            if view_img:
                cv2.imshow(str(p), im0)
                cv2.waitKey(1)  # 1 millisecond

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'image':
                    cv2.imwrite(save_path, im0)
                else:  # 'video' or 'stream'
                    if vid_path != save_path:  # new video
                        vid_path = save_path
                        if isinstance(vid_writer, cv2.VideoWriter):
                            vid_writer.release()  # release previous video writer
                        if vid_cap:  # video
                            fps = vid_cap.get(cv2.CAP_PROP_FPS)
                            w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                            h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        else:  # stream
                            fps, w, h = 30, im0.shape[1], im0.shape[0]
                            save_path += '.mp4'
                        vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
                    vid_writer.write(im0)

    if save_txt or save_img:
        s = f"\n{len(list(save_dir.glob('labels/*.txt')))} labels saved to {save_dir / 'labels'}" if save_txt else ''
        print(f"Results saved to {save_dir}{s}")

    if update:
        strip_optimizer(weights)  # update model (to fix SourceChangeWarning)

    print(f'Done. ({time.time() - t0:.3f}se)')

In [28]:
def parse_opt():
    parser = argparse.ArgumentParser()
    parser.add_argument('--weights', nargs='+', type=str, default='yolov5s_jihyun.pt', help='model.pt path(s)')
    parser.add_argument('--source', type=str, default='/home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2582.MOV', help='file/dir/URL/glob, 0 for webcam')
    parser.add_argument('--imgsz', '--img', '--img-size', type=int, default=640, help='inference size (pixels)')
    parser.add_argument('--conf-thres', type=float, default=0.25, help='confidence threshold')
    parser.add_argument('--iou-thres', type=float, default=0.45, help='NMS IoU threshold')
    parser.add_argument('--max-det', type=int, default=1000, help='maximum detections per image')
    parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')conda conda install
    parser.add_argument('--view-img', action='store_true', help='show results')
    parser.add_argument('--save-txt', action='store_true', help='save results to *.txt')
    parser.add_argument('--save-conf', action='store_true', help='save confidences in --save-txt labels')
    parser.add_argument('--save-crop', action='store_true', help='save cropped prediction boxes')
    parser.add_argument('--nosave', action='store_true', help='do not save images/videos')
    parser.add_argument('--classes', nargs='+', type=int, help='filter by class: --class 0, or --class 0 2 3')
    parser.add_argument('--agnostic-nms', action='store_true', help='class-agnostic NMS')
    parser.add_argument('--augment', action='store_true', help='augmented inference')
    parser.add_argument('--update', action='store_true', help='update all models')
    parser.add_argument('--project', default='runs/detect', help='save results to project/name')
    parser.add_argument('--name', default='exp', help='save results to project/name')
    parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
    parser.add_argument('--line-thickness', default=3, type=int, help='bounding box thickness (pixels)')
    parser.add_argument('--hide-labels', default=False, action='store_true', help='hide labels')
    parser.add_argument('--hide-conf', default=False, action='store_true', help='hide confidences')
    parser.add_argument('--half', action='store_true', help='use FP16 half-precision inference')
    opt, unk = parser.parse_known_args()
    return opt


def main(opt):
    print(colorstr('detect: ') + ', '.join(f'{k}={v}' for k, v in vars(opt).items()))
    check_requirements(exclude=('tensorboard', 'thop'))
    run(**vars(opt))


if __name__ == "__main__":
    opt = parse_opt()
    main(opt)

YOLOv5 🚀 2021-6-22 torch 1.8.1 CPU

Fusing layers... 


detect: weights=yolov5s_jihyun.pt, source=/home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV, imgsz=640, conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
requirements: /home/piai/바탕화면/YOLO_trial/requirements.txt not found, check failed.


Model Summary: 224 layers, 7053910 parameters, 0 gradients, 16.3 GFLOPs


video 1/1 (1/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.195s)
video 1/1 (2/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.193s)
video 1/1 (3/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.198s)
video 1/1 (4/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.204s)
video 1/1 (5/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.238s)
video 1/1 (6/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.199s)
video 1/1 (7/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.198s)
video 1/1 (8/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.192s)
video 1/1 (9/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.185s)
video 1/1 (10/473) 

video 1/1 (149/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.189s)
video 1/1 (150/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.180s)
video 1/1 (151/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.191s)
video 1/1 (152/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.185s)
video 1/1 (153/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.200s)
video 1/1 (154/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.182s)
video 1/1 (155/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.195s)
video 1/1 (156/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.186s)
video 1/1 (157/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.180s)
v

video 1/1 (223/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.203s)
video 1/1 (224/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.192s)
video 1/1 (225/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.194s)
video 1/1 (226/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.190s)
video 1/1 (227/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.182s)
video 1/1 (228/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.187s)
video 1/1 (229/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.197s)
video 1/1 (230/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.184s)
video 1/1 (231/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.184s)
v

video 1/1 (297/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.193s)
video 1/1 (298/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.179s)
video 1/1 (299/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.184s)
video 1/1 (300/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.196s)
video 1/1 (301/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.183s)
video 1/1 (302/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.182s)
video 1/1 (303/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.183s)
video 1/1 (304/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.199s)
video 1/1 (305/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.202s)
v

video 1/1 (371/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.189s)
video 1/1 (372/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.192s)
video 1/1 (373/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.188s)
video 1/1 (374/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.186s)
video 1/1 (375/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.196s)
video 1/1 (376/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.189s)
video 1/1 (377/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.199s)
video 1/1 (378/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.185s)
video 1/1 (379/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.190s)
v

video 1/1 (445/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.195s)
video 1/1 (446/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.182s)
video 1/1 (447/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.181s)
video 1/1 (448/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.197s)
video 1/1 (449/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.187s)
video 1/1 (450/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.203s)
video 1/1 (451/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.185s)
video 1/1 (452/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.199s)
video 1/1 (453/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.192s)
v

In [28]:
def parse_opt():
    parser = argparse.ArgumentParser()
    parser.add_argument('--weights', nargs='+', type=str, default='yolov5s_jihyun.pt', help='model.pt path(s)')
    parser.add_argument('--source', type=str, default=0, help='file/dir/URL/glob, 0 for webcam')
    parser.add_argument('--imgsz', '--img', '--img-size', type=int, default=640, help='inference size (pixels)')
    parser.add_argument('--conf-thres', type=float, default=0.25, help='confidence threshold')
    parser.add_argument('--iou-thres', type=float, default=0.45, help='NMS IoU threshold')
    parser.add_argument('--max-det', type=int, default=1000, help='maximum detections per image')
    parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')conda conda install
    parser.add_argument('--view-img', action='store_true', help='show results')
    parser.add_argument('--save-txt', action='store_true', help='save results to *.txt')
    parser.add_argument('--save-conf', action='store_true', help='save confidences in --save-txt labels')
    parser.add_argument('--save-crop', action='store_true', help='save cropped prediction boxes')
    parser.add_argument('--nosave', action='store_true', help='do not save images/videos')
    parser.add_argument('--classes', nargs='+', type=int, help='filter by class: --class 0, or --class 0 2 3')
    parser.add_argument('--agnostic-nms', action='store_true', help='class-agnostic NMS')
    parser.add_argument('--augment', action='store_true', help='augmented inference')
    parser.add_argument('--update', action='store_true', help='update all models')
    parser.add_argument('--project', default='runs/detect', help='save results to project/name')
    parser.add_argument('--name', default='exp', help='save results to project/name')
    parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
    parser.add_argument('--line-thickness', default=3, type=int, help='bounding box thickness (pixels)')
    parser.add_argument('--hide-labels', default=False, action='store_true', help='hide labels')
    parser.add_argument('--hide-conf', default=False, action='store_true', help='hide confidences')
    parser.add_argument('--half', action='store_true', help='use FP16 half-precision inference')
    opt, unk = parser.parse_known_args()
    return opt


def main(opt):
    print(colorstr('detect: ') + ', '.join(f'{k}={v}' for k, v in vars(opt).items()))
    check_requirements(exclude=('tensorboard', 'thop'))
    run(**vars(opt))


if __name__ == "__main__":
    opt = parse_opt()
    main(opt)

YOLOv5 🚀 2021-6-22 torch 1.8.1 CPU

Fusing layers... 


detect: weights=yolov5s_jihyun.pt, source=/home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV, imgsz=640, conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
requirements: /home/piai/바탕화면/YOLO_trial/requirements.txt not found, check failed.


Model Summary: 224 layers, 7053910 parameters, 0 gradients, 16.3 GFLOPs


video 1/1 (1/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.195s)
video 1/1 (2/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.193s)
video 1/1 (3/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.198s)
video 1/1 (4/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.204s)
video 1/1 (5/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.238s)
video 1/1 (6/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.199s)
video 1/1 (7/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.198s)
video 1/1 (8/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.192s)
video 1/1 (9/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.185s)
video 1/1 (10/473) 

video 1/1 (149/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.189s)
video 1/1 (150/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.180s)
video 1/1 (151/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.191s)
video 1/1 (152/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.185s)
video 1/1 (153/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.200s)
video 1/1 (154/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.182s)
video 1/1 (155/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.195s)
video 1/1 (156/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.186s)
video 1/1 (157/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.180s)
v

video 1/1 (223/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.203s)
video 1/1 (224/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.192s)
video 1/1 (225/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.194s)
video 1/1 (226/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.190s)
video 1/1 (227/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.182s)
video 1/1 (228/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.187s)
video 1/1 (229/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.197s)
video 1/1 (230/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.184s)
video 1/1 (231/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.184s)
v

video 1/1 (297/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.193s)
video 1/1 (298/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.179s)
video 1/1 (299/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.184s)
video 1/1 (300/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.196s)
video 1/1 (301/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.183s)
video 1/1 (302/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.182s)
video 1/1 (303/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.183s)
video 1/1 (304/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.199s)
video 1/1 (305/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.202s)
v

video 1/1 (371/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.189s)
video 1/1 (372/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.192s)
video 1/1 (373/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.188s)
video 1/1 (374/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.186s)
video 1/1 (375/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.196s)
video 1/1 (376/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.189s)
video 1/1 (377/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.199s)
video 1/1 (378/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.185s)
video 1/1 (379/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.190s)
v

video 1/1 (445/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.195s)
video 1/1 (446/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.182s)
video 1/1 (447/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.181s)
video 1/1 (448/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.197s)
video 1/1 (449/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.187s)
video 1/1 (450/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.203s)
video 1/1 (451/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.185s)
video 1/1 (452/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.199s)
video 1/1 (453/473) /home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV: 640x384 1 Home_Point, Done. (0.192s)
v

!python detect_jihyun.py --weight './runs/train/exp/weights/best.pt' --source '/home/piai/바탕화면/YOLO trial(1)/VideoFile/IMG_2581.MOV' --conf 0.6

import torchvision
torchvision.__version__